### Banco maquina local

Nesse notebook, ao invez de utilizar uma VPS da AWS, esse banco está instalado em uma maquina física

In [5]:
#importações

import pandas as pd
import pyodbc
from datetime import datetime
from pegar_historico import pegar_historico
from dotenv import load_dotenv
import os
from datetime import date

In [7]:


#Parametros para estabecer a conexão com o banco
load_dotenv() 

connection_string = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={os.getenv('DB_HOST')},{os.getenv('DB_PORT')};
    DATABASE={os.getenv('DB_NAME')};
    UID={os.getenv('DB_USER')};
    PWD={os.getenv("DB_PASSWORD")};
    Encrypt=yes;
    TrustServerCertificate=yes;
"""

## Todavia é interessante criar um banco e tabela nova utilizando essa conexao 

In [20]:
# Conectando ao servidor, usando o banco 'master' (obrigatório para criar novos databases)

# Abrir conexão
conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()

# Nome do novo banco de dados
new_db_name = 'bitcoin_stats'

# Comando para criar o banco
create_db_query = f"CREATE DATABASE {new_db_name};"

try:
    cursor.execute(create_db_query)
    print(f"Banco de dados '{new_db_name}' criado com sucesso!")
except Exception as e:
    print("Erro ao criar banco de dados:", e)

# Fechar conexão
cursor.close()
conn.close()


Banco de dados 'bitcoin_stats' criado com sucesso!


## Checando se o novo banco foi criado

In [43]:
#Primeiro, checo quais tabelas já existem no banco
def checa_bancos_existentes():
    conn = pyodbc.connect(connection_string, autocommit=True)
    #Query para checar os bancos existentes
    query = """
    SELECT 
        name
    FROM sys.databases; 
    """
    
    df = pd.read_sql(query, conn)
    
    conn.close()
    
    return df

checa_bancos_existentes()

C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_22412\1738049834.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,name
0,master
1,tempdb
2,model
3,msdb
4,DW_SUCOS
5,bitcoin_stats


In [53]:
#Crio a nova tabela
conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()

crate_table_query = """
CREATE TABLE bitcoin_stats.dbo.bitcoin_prices(
    id INT IDENTITY(1,1) PRIMARY KEY,
    symbol VARCHAR(10),
    timestamp DATETIME,
    [open] DECIMAL(18, 8),
    [high] DECIMAL(18, 8),
    [low] DECIMAL(18, 8),
    [close] DECIMAL(18, 8),
    volume DECIMAL(18, 8)
);
"""

try:
    cursor.execute(crate_table_query)
    print("Tabela crada com sucesso")
except Exception as e:
    print("Erro ao criar tabela", e)

cursor.close()
conn.close()

Tabela crada com sucesso


## Checando agora se a tabela foi criada

In [55]:
#Primeiro, checo quais tabelas já existem no banco
def checa_tabelas_existentes():
    conn = pyodbc.connect(connection_string, autocommit=True)
    #Query para checar os bancos existentes
    query = """
    SELECT 
        TABLE_NAME
    FROM bitcoin_stats.INFORMATION_SCHEMA.TABLES; 
    """
    
    df = pd.read_sql(query, conn)
    
    conn.close()
    
    return df

checa_tabelas_existentes()

C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_22412\3350115091.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,TABLE_NAME
0,bitcoin_prices


## Criando o dataframe

In [105]:
par = "BTCBRL"
intervalo = "1d"
data_inicio = "1 Jan 2025"

df_btcbrl = pegar_historico(
        par,
        intervalo,
        data_inicio
    )

df_btcbrl.head(5)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2025-01-01,583135.0,591233.0,579057.0,588693.0,72.14957,1735775999999,42310167.57540000,21358,41.00811000,24046544.55393000,0
1,2025-01-02,588635.0,607111.0,587263.0,599306.0,166.56756,1735862399999,99793602.55375000,47531,78.05491000,46770433.42530000,0
2,2025-01-03,599363.0,613793.0,594140.0,607582.0,128.54397,1735948799999,77592339.31612000,39792,61.49631000,37100120.06508000,0
3,2025-01-04,607593.0,612902.0,604369.0,609413.0,64.64526,1736035199999,39296764.14958000,27377,31.38893000,19078907.64598000,0
4,2025-01-05,609412.0,613000.0,604300.0,611273.0,58.20556,1736121599999,35453798.90673000,22973,27.81086000,16944045.55734000,0


In [107]:
columns = df_btcbrl.columns[0:6]

df_btcbrl = df_btcbrl[columns]

df_btcbrl['symbol'] = 'btc'

df_btcbrl

,timestamp,open,high,low,close,volume,symbol
0,2025-01-01,583135.0,591233.0,579057.0,588693.0,72.14957,btc
1,2025-01-02,588635.0,607111.0,587263.0,599306.0,166.56756,btc
2,2025-01-03,599363.0,613793.0,594140.0,607582.0,128.54397,btc
3,2025-01-04,607593.0,612902.0,604369.0,609413.0,64.64526,btc
4,2025-01-05,609412.0,613000.0,604300.0,611273.0,58.20556,btc
...,...,...,...,...,...,...,...
176,2025-06-26,599442.0,603221.0,588752.0,590103.0,143.57173,btc
177,2025-06-27,590131.0,594507.0,585054.0,591150.0,103.22926,btc
178,2025-06-28,591237.0,594409.0,589963.0,592820.0,33.27928,btc
179,2025-06-29,592820.0,598520.0,591501.0,597527.0,52.20595,btc


In [121]:
#cria uma lista com tuplas
data_to_insert = [
    (
        row.symbol,
        row.timestamp, 
        row.open, 
        row.high, 
        row.low, 
        row.close, 
        row.volume
        
    )
    for _, row in df_btcbrl.iterrows()
]

In [123]:
conn = pyodbc.connect(connection_string)

#Construindo a query de inserção
insert_query = """
    INSERT INTO bitcoin_stats.dbo.bitcoin_prices VALUES (?, ?, ?, ?, ?, ?, ?)
"""

cursor =  conn.cursor()

#O executemany ja itera atraves dos item da lista
cursor.executemany(insert_query, data_to_insert)
conn.commit()
conn.close()

In [137]:
conn = pyodbc.connect(connection_string)

query = "SELECT * FROM bitcoin_stats.dbo.bitcoin_prices"

df = pd.read_sql(query, conn)

conn.close()

df



C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_22412\3326392694.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,symbol,timestamp,open,high,low,close,volume
0,2,btc,2025-01-01,583135.0,591233.0,579057.0,588693.0,72.14957
1,3,btc,2025-01-02,588635.0,607111.0,587263.0,599306.0,166.56756
2,4,btc,2025-01-03,599363.0,613793.0,594140.0,607582.0,128.54397
3,5,btc,2025-01-04,607593.0,612902.0,604369.0,609413.0,64.64526
4,6,btc,2025-01-05,609412.0,613000.0,604300.0,611273.0,58.20556
...,...,...,...,...,...,...,...,...
176,178,btc,2025-06-26,599442.0,603221.0,588752.0,590103.0,143.57173
177,179,btc,2025-06-27,590131.0,594507.0,585054.0,591150.0,103.22926
178,180,btc,2025-06-28,591237.0,594409.0,589963.0,592820.0,33.27928
179,181,btc,2025-06-29,592820.0,598520.0,591501.0,597527.0,52.20595
